<a href="https://colab.research.google.com/github/YagyanshB/SemEval-Task6-CS408/blob/main/Convolution_Neural_Network_Baseline_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of Convolutional Neural Network:

---



In [2]:
import keras
keras.__version__

'2.4.3'

In [3]:
# IMPORT MODULES
import numpy as np 
import re
import pandas as pd 

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Embedding 
from keras.layers import concatenate 
from keras.layers import Input 
from keras.layers import SpatialDropout1D 
from keras.layers import Dropout 
from keras.layers import GRU 
from keras.layers import Bidirectional 
from keras.layers import Flatten
from keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [1]:
# INITIALIZATION

train_file    = 'drive/My Drive/olid-training-v1.0.tsv' 

test_file_a   = 'drive/My Drive/testset-levela.tsv' 
test_file_b   = 'drive/My Drive/testset-levelb.tsv' # set corresponding path for the test dataset file
test_file_c   = 'drive/My Drive/testset-levelc.tsv' # set corresponding path for the test dataset file

test_labels_a = 'drive/My Drive/labels-levela.csv' 
test_labels_b = 'drive/My Drive/labels-levelb.csv' # set corresponding path for the labels of the test dataset file
test_labels_c = 'drive/My Drive/labels-levelc.csv' # set corresponding path for the labels of the test dataset file


maxlen = 200
max_fatures = 10000

# GloVe Pretrained Embeddings


embedding_dim = 200

models_a = list()
models_b = list()
models_c = list()
ensemble_models_a = list()
ensemble_models_b = list()
ensemble_models_c = list()

In [34]:
# UTIL FUNCTIONS

def replace_hashtag(tweet):
  hashtags = find_hashtag(tweet)
  for hashtag in hashtags:
    split = split_hashtag(hashtag)
    tweet = tweet.replace(hashtag, split)
  tweet = tweet.replace('#', '')
  return tweet

def find_hashtag(tweet):
  hashtags = re.findall(r"#(\w+)", tweet)
  return hashtags

def split_hashtag(hashtag):
  fo = re.compile(r'#[A-Z]{2,}(?![a-z])|[A-Z][a-z]+')
  fi = fo.findall(hashtag)
  result = ''
  for var in fi:
    result += var + ' '
  return result

# TRAINING DATA (FOR ALL THE SUBTASKS)
def import_data(train_file, test_file_a, test_labels_a):
  # IMPORT DATASET
  data = pd.read_csv(train_file, sep='\t', header=0)
  data = data[['id','tweet', 'subtask_a', 'subtask_b', 'subtask_c']]

  # DATA PREPROCESSING
  data_preprocessing(data)

  # REAL TEST DATASET
  data_test = pd.read_csv(test_file, sep='\t', header=0)

  # DATA PREPROCESSING
  # split hashtags into words

  for i, j in data_test.iterrows():
    data_test.at[i,'tweet'] = replace_hashtag(j['tweet'])
  data_test['tweet'] = data_test['tweet'].apply(lambda x: x.lower()) # lowercase
  data_test['tweet'] = data_test['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) # remove special characters
  data_test['tweet'] = data_test['tweet'].str.replace('user','') # remove 'user' tokens
  data_test['tweet'] = data_test['tweet'].str.replace('url','') # remove 'url' tokens
  labels_test = pd.read_csv(test_labels, sep=',', header=0)
  labels_test = labels_test[['id', subtask_name]]
  data_test = pd.merge(data_test, labels_test, on='id')
  Y = pd.get_dummies(data[subtask_name]).values

  # TOKENIZER

  tokenizer = Tokenizer(num_words=max_fatures, split=' ')
  tokenizer.fit_on_texts(data['tweet'].values)
  X = tokenizer.texts_to_sequences(data['tweet'].values)
  X = pad_sequences(X, maxlen=maxlen)

# DATA PREPROCESSING STEPS

def data_preprocessing(data):
  for i, j in data.iterrows():
    data.at[i,'tweet'] = replace_hashtag(j['tweet']) # split hashtags into words
  data['tweet'] = data['tweet'].apply(lambda x: x.lower()) # lowercase
  data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) # remove special characters
  data['tweet'] = data['tweet'].str.replace('user','') # remove 'user' tokens
  data['tweet'] = data['tweet'].str.replace('url','') # remove 'url' tokens

  # Testing with validation dataset
  X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)

  # Testing with original test dataset
  X_test_real = tokenizer.texts_to_sequences(data_test['tweet'].values)
  X_test_real = pad_sequences(X_test_real, maxlen=maxlen)
  Y_test_real = pd.get_dummies(data_test[subtask_name]).values
  
  return X_train, X_test, Y_train, Y_test, X_test_real, Y_test_real, tokenizer
# SPLIT HASHTAG INTO WORDS UTIL FUNCTIONS


In [35]:

def pretrained_embeddings(tokeniser):
  embedding_file = open('drive/My Drive/glove.twitter.27B.100d.txt.gz', encoding = 'utf-8')

  embeddings_index = {}
  
  for line in embedding_file:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  embedding_file.close()

  print('Found %s word vectors.' % len(embeddings_index))
  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  embedding_matrix = np.zeros((max_fatures, embedding_dim))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if i < max_fatures:
          if embedding_vector is not None:
              # Words not found in embedding index will be all-zeros.
             embedding_matrix[i] = embedding_vector
  return embedding_matrix

In [36]:
def prediction_and_results(model, X_test, Y_test, X_test_real, Y_test_real):

  # PREDICTION USING VALIDATION DATASET
  print('Model:', model_name)
  print('Model prediction with validation dataset:')
  
  # Prediction for class Model
  Y_pred = model.predict(X_test,batch_size = batch_size)
  Y_pred = np.argmax(Y_pred,axis=1)
  df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred': Y_pred})
  df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
  print("Confusion matrix:", confusion_matrix(df_test.true, df_test.pred))
  print(classification_report(df_test.true, df_test.pred, digits=4))

  # PREDICTION USING REAL TEST DATASET
  print('Model:', model_name)
  print('Model prediction with test dataset:')

  # Prediction for class Model
  Y_pred_real = model.predict(X_test_real,batch_size = batch_size)
  Y_pred_real = np.argmax(Y_pred_real,axis=1)
  df_test_real = pd.DataFrame({'true': Y_test_real.tolist(), 'pred':Y_pred_real})
  df_test_real['true'] = df_test_real['true'].apply(lambda x: np.argmax(x))
  print("Confusion matrix", confusion_matrix(df_test_real.true, df_test_real.pred))
  
  # PREDICTION FOR ONE MODEL
  print(classification_report(df_test_real.true, df_test_real.pred, digits=4))

In [37]:
def prediction_and_results_ensemble(ensemble_model, X_test, Y_test, X_test_real, Y_test_real):


  y_combine = [model.predict(X_test, batch_size = batch_size) for model in ensemble_model]
  y_combine = np.array(y_combine)

  # sum across ensembles
  summed = np.sum(y_combine, axis=0)

  # argmax across classes
  Y_pred = np.argmax(summed, axis=1)
  df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred': Y_pred})
  df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
  print("Confusion matrix:", confusion_matrix(df_test.true, df_test.pred))
  print(classification_report(df_test.true, df_test.pred, digits=4))

  # PREDICTION USING REAL TEST DATASET
  y_combine = [model.predict(X_test_real, batch_size = batch_size) for model in ensemble_model]
  y_combine = np.array(y_combine)

  # sum across ensembles
  summed = np.sum(y_combine, axis=0)
  
  # argmax across classes
  Y_pred_real = np.argmax(summed, axis=1)
  df_test_real = pd.DataFrame({'true': Y_test_real.tolist(), 'pred':Y_pred_real})
  df_test_real['true'] = df_test_real['true'].apply(lambda x: np.argmax(x))
  print("Confusion matrix", confusion_matrix(df_test_real.true, df_test_real.pred))
  print(classification_report(df_test_real.true, df_test_real.pred, digits=4))

In [31]:
def initialize_CNN_GMP(output):
  model_name = 'CNN + GLOBAL MAX POOLING'
  model = Sequential()
  #model.add(Embedding(max_fatures, embedding_dim, input_length = maxlen, trainable = False)) #without pretrained embeddings
  model.add(Embedding(max_fatures, embedding_dim, weights = [embedding_matrix], input_length=maxlen, trainable=True)) #with pretr
  model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
  model.add(GlobalMaxPooling1D())
  # CNN + GLOBAL MAX POOLING
  model.add(Dense(256, activation='relu'))
  model.add(Dense(output, activation='softmax'))
  return model, model_name

In [ ]:
def initialize_CNN_3(output):
  model_name = 'CNN'
  embedding_dim = 200
  i = Input(shape=(maxlen,), dtype='int32', name='main_input')
  x = Embedding(max_fatures, embedding_dim ,weights = [embedding_matrix], input_length=maxlen, trainable=True)(i)
  x = Dropout(0.4)(x)
  def get_conv_pool(x_input, max_len, sufix, n_grams=[2,3,4], feature_maps=256):
        branches = []
        for n in n_grams:
            branch = Conv1D(filters=feature_maps, kernel_size=n, activation='relu', name='Conv_'+sufix+'_'+str(n))(x_input)
            branch = MaxPooling1D(pool_size=max_len-n+1, strides=None, padding='valid', name='MaxPooling_'+sufix+'_'+str(n))(bran
            branch = Flatten(name='Flatten_'+sufix+'_'+str(n))(branch)
            branches.append(branch)
        return branches
  branches = get_conv_pool(x, maxlen, 'dynamic')
  z = concatenate(branches, axis=-1)
  z1 = Dropout(0.3)(z)
  z2 = Dense(256, activation='relu')(z1)
  o = Dense(2, activation='softmax')(z2)
  # CNN (3 filters)
  model = Model(inputs=i, outputs=o)
  return model, model_name

In [ ]:
# SUBTASK A
# IMPORT DATA
print('IMPORT DATA...')
X_train, X_test_a, Y_train_a, Y_test_a, X_test_real_a, Y_test_real_a, tokenizer = import_data(train_file, test_file_a, test_labels_a)
embedding_matrix = pretrained_embeddings(tokenizer)

# For Different SUBTASKS B & C, all we need to do is change the test_file_a & test_labels_a to either
# test_file_b or test_file_c and test_labels_b or test_labels_c

In [1]:
# INITIALIZING OUR CNN MODELS

model_name = initialize_CNN_GMP(2)
model_name = initialize_CNN_3(2)

In [ ]:
print('COMPILING MODEL')
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
# MODEL SCHEME
print('PLOT THE MODEL...')
print('Model:', model_name)
print(model.summary())
plot_model(model, show_shapes=True, show_layer_names=True)

# SUBTASK A: FIT THE MODEL
print('FIT THE MODEL...')
print('Model:', model_name)
batch_size = 128
model.fit(X_train, Y_train_a, epochs = 10, batch_size=batch_size)